###사전 설정

In [1]:
!pip install --upgrade pip
!pip install torch torchvision torchaudio
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install scipy pillow transformers

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-8pkggqaq
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-8pkggqaq
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=f24f3a0d9dd3f4cac3781e01c64178fc1b90afea99ba3a088e0b929fd280bd32
  Stored in directory: /tmp/pip-ephem-wheel-cache-997hrnag/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip
Using device: cuda


In [26]:
import numpy as np
import matplotlib.pyplot as plt
from  tqdm import tqdm
import random
import torch
import torchvision
import torchvision.transforms as T
from torchvision.transforms import ToPILImage
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
import os
from torchvision import transforms
import scipy.io
import shutil
import clip
from PIL import Image
import pandas as pd

###wikiart

In [3]:
import kagglehub

path = kagglehub.dataset_download("steubk/wikiart")

print("Path to dataset files:", path)

100%|██████████| 31.4G/31.4G [04:15<00:00, 132MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/steubk/wikiart/versions/1


In [4]:
!mv "{path}" /content/

In [5]:
#데이터셋 분리 코드
csv_path = "/content/dataset/classes.csv"
data = pd.read_csv(csv_path)

source_folder = "/content/dataset"

train_folder = "/content/dataset/train"
test_folder = "/content/dataset/test"
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)


def extract_class_and_filename(row):
    full_path = row['filename']
    class_name = full_path.split('/')[0]
    filename = os.path.basename(full_path)
    return class_name, filename

def move_and_rename_files(row, target_folder):
    class_name, filename = extract_class_and_filename(row)
    source_path = os.path.join(source_folder, class_name, filename)
    target_filename = f"{class_name}_{row.name}.jpg"
    target_path = os.path.join(target_folder, target_filename)

    if os.path.exists(source_path):
        shutil.move(source_path, target_path)
    else:
        print(f"File not found: {source_path}")

train_data = data[data['subset'] == 'train']
train_data.apply(lambda row: move_and_rename_files(row, train_folder), axis=1)

test_data = data[data['subset'] == 'test']
test_data.apply(lambda row: move_and_rename_files(row, test_folder), axis=1)

print("Dataset splitting and renaming completed.")

File not found: /content/dataset/Art_Nouveau_Modern/eugã¨ne-grasset_a-la-place-clichy.jpg
File not found: /content/dataset/Art_Nouveau_Modern/eugã¨ne-grasset_abricotine.jpg
File not found: /content/dataset/Art_Nouveau_Modern/eugã¨ne-grasset_affiche-pour-the-century-magazine-napol-on.jpg
File not found: /content/dataset/Art_Nouveau_Modern/eugã¨ne-grasset_anxi-t-1897.jpg
File not found: /content/dataset/Art_Nouveau_Modern/eugã¨ne-grasset_bonne-nouvelle.jpg
File not found: /content/dataset/Art_Nouveau_Modern/eugã¨ne-grasset_encre-l-marquet.jpg
File not found: /content/dataset/Art_Nouveau_Modern/eugã¨ne-grasset_esclarmonde.jpg
File not found: /content/dataset/Art_Nouveau_Modern/eugã¨ne-grasset_froideur.jpg
File not found: /content/dataset/Art_Nouveau_Modern/eugã¨ne-grasset_girl-in-the-garden.jpg
File not found: /content/dataset/Art_Nouveau_Modern/eugã¨ne-grasset_grafton-gallery-from-les-affiche-illustrees-1897.jpg
File not found: /content/dataset/Art_Nouveau_Modern/eugã¨ne-grasset_harper-s

In [14]:
test_folder = "/content/dataset/train"
test_file_count = len([file for file in os.listdir(test_folder) if os.path.isfile(os.path.join(test_folder, file))])

test_file_count

63443

In [7]:
# clip 전처리 코드
device = "cuda" if torch.cuda.is_available() else "cpu"
_, preprocess = clip.load("ViT-B/32", device=device)

source_folder = "/content/dataset/test"
processed_folder = "/content/dataset/processed_dataset"
os.makedirs(processed_folder, exist_ok=True)

print("Preprocessing and saving images...")
for root, _, files in os.walk(source_folder):
    for file in tqdm(files):
        if file.endswith(".jpg") or file.endswith(".png"):
            source_path = os.path.join(root, file)

            relative_path = os.path.relpath(root, source_folder)
            dest_folder = os.path.join(processed_folder, relative_path)
            os.makedirs(dest_folder, exist_ok=True)

            dest_path = os.path.join(dest_folder, file)

            try:
                image = Image.open(source_path).convert("RGB")
                processed_image = preprocess(image)

                torch.save(processed_image, dest_path + ".pt")
            except Exception as e:
                print(f"Error processing file {source_path}: {e}")

print(f"All images have been preprocessed and saved to {processed_folder}.")

100%|███████████████████████████████████████| 338M/338M [00:05<00:00, 61.3MiB/s]


Preprocessing and saving images...


100%|██████████| 15860/15860 [10:58<00:00, 24.09it/s]

All images have been preprocessed and saved to /content/dataset/processed_dataset.


In [18]:
#png파일 전환 코드
tensor_folder = "/content/dataset/preprocessed_train"
output_folder = "/content/dataset/png_train"
os.makedirs(output_folder, exist_ok=True)

print("Converting tensors to PNG images...")
for root, _, files in os.walk(tensor_folder):
    for file in tqdm(files):
        if file.endswith(".pt"):

            tensor_path = os.path.join(root, file)
            tensor = torch.load(tensor_path)

            to_pil = ToPILImage()
            image = to_pil(tensor)

            relative_path = os.path.relpath(root, tensor_folder)
            dest_folder = os.path.join(output_folder, relative_path)
            os.makedirs(dest_folder, exist_ok=True)
            output_path = os.path.join(dest_folder, file.replace(".pt", ".png"))

            image.save(output_path)

print(f"All tensors have been converted and saved to {output_folder}.")

Converting tensors to PNG images...


  0%|          | 0/7482 [00:00<?, ?it/s]<ipython-input-18-95373c91265f>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensor = torch.load(tensor_path)
100%|██████████| 7

All tensors have been converted and saved to /content/dataset/png_train.


In [19]:
# zip 압축 코드
folder_to_zip = "/content/dataset/png_train"
output_zip_file = "/content/dataset/png_LP_train.zip"

shutil.make_archive(output_zip_file.replace('.zip', ''), 'zip', folder_to_zip)
print(f"Folder compressed to {output_zip_file}")

Folder compressed to /content/dataset/png_LP_train.zip


In [20]:
# Google Drive 마운트
!cp /content/dataset/png_LP_train.zip /content/drive/MyDrive/
print("File copied to Google Drive!")

File copied to Google Drive!


In [16]:
# clip 전처리 후 png전환 코드
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

source_folder = "/content/dataset/train"
destination_folder = "/content/dataset/preprocessed_train"
os.makedirs(destination_folder, exist_ok=True)

num_images_per_class = 300  #클래스별 이미지 수

class_images = {}
for file in os.listdir(source_folder):
    if file.endswith(".jpg"):
        class_name = file.split("_")[0]
        if class_name not in class_images:
            class_images[class_name] = []
        class_images[class_name].append(file)

print("Selecting and preprocessing images...")
for class_name, images in tqdm(class_images.items()):
    selected_images = random.sample(images, min(len(images), num_images_per_class))

    class_folder = os.path.join(destination_folder, class_name)
    os.makedirs(class_folder, exist_ok=True)

    for image_name in selected_images:
        source_path = os.path.join(source_folder, image_name)
        dest_path = os.path.join(destination_folder, image_name.replace(".jpg", ".pt"))

        try:
            image = Image.open(source_path).convert("RGB")
            processed_tensor = preprocess(image)
            torch.save(processed_tensor, dest_path)
        except Exception as e:
            print(f"Error processing {source_path}: {e}")

print(f"Preprocessed images saved to {destination_folder}.")

Selecting and preprocessing images...


100%|██████████| 27/27 [05:10<00:00, 11.52s/it]

Preprocessed images saved to /content/dataset/preprocessed_train.


###wikiart zeroshot

In [59]:
# clip 제로샷 코드
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

class TensorDataset(Dataset):
    def __init__(self, tensor_folder, class_names):
        self.tensor_folder = tensor_folder
        self.class_names = class_names
        self.tensor_files = [os.path.join(tensor_folder, file) for file in os.listdir(tensor_folder) if file.endswith(".pt")]

    def __len__(self):
        return len(self.tensor_files)

    def __getitem__(self, idx):
        tensor_path = self.tensor_files[idx]
        tensor = torch.load(tensor_path)
        label = os.path.basename(tensor_path).split("_")[0]
        return tensor, label

tensor_folder = "/content/dataset/pre_train"
class_names = [
    "Abstract Expressionism", "Action painting", "Analytical Cubism", "Art Nouveau Modern", "Baroque", "Color Field Painting",
    "Contemporary Realism", "Cubism", "Early Renaissance", "Expressionism", "Fauvism", "High Renaissance",
    "Impressionism", "Mannerism Late Renaissance", "Minimalism", "Naive Art Primitivism", "New Realism",
    "Northern Renaissance", "Pointillism", "Pop Art", "Post Impressionism", "Realism", "Rococo", "Romanticism",
    "Symbolism", "Synthetic Cubism", "Ukiyo_e"
]

dataset = TensorDataset(tensor_folder, class_names)

batch_size = 1024  #배치 사이즈
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

print("Caching text embeddings...")
text_prompts = [f"An artwork in the style of {class_name}" for class_name in class_names]
text_inputs = clip.tokenize(text_prompts).to(device)
with torch.no_grad():
    text_features = model.encode_text(text_inputs)
    text_features /= text_features.norm(dim=-1, keepdim=True)

correct = 0
total = 0

print("Evaluating zero-shot classification...")
with torch.no_grad():
    for batch in tqdm(dataloader):
        images, labels = batch
        images = images.to(device)

        image_features = model.encode_image(images)
        image_features /= image_features.norm(dim=-1, keepdim=True)

        similarity = (image_features @ text_features.T).softmax(dim=-1)

        predictions = similarity.argmax(dim=-1).cpu().numpy()
        total += len(labels)

        for label, pred in zip(labels, predictions):
            if class_names[pred].replace(" ", "_") == label:
                correct += 1

accuracy = correct / total * 100
print(f"Zero-shot CLIP classification accuracy: {accuracy:.2f}%")

Caching text embeddings...
Evaluating zero-shot classification...


  0%|          | 0/8 [00:00<?, ?it/s]<ipython-input-59-e2c0687df371>:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensor = torch.load(tensor_path)  # Load preprocessed 

Zero-shot CLIP classification accuracy: 10.38%


###wikiart linear probe

In [73]:
#로지스틱 회귀 적합(LP)
import os
import torch
from torch.utils.data import DataLoader, Dataset
import clip
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

class TensorDataset(Dataset):
    def __init__(self, tensor_folder, class_names):
        self.tensor_folder = tensor_folder
        self.class_names = class_names
        self.tensor_files = [os.path.join(tensor_folder, file) for file in os.listdir(tensor_folder) if file.endswith(".pt")]

    def __len__(self):
        return len(self.tensor_files)

    def __getitem__(self, idx):
        tensor_path = self.tensor_files[idx]
        tensor = torch.load(tensor_path)
        label = self.class_names.index(" ".join(os.path.basename(tensor_path).split("_")[:-1]))
        return tensor, label

tensor_folder = "/content/dataset/pre_train"
class_names = [
    "Abstract Expressionism", "Action painting", "Analytical Cubism", "Art Nouveau Modern", "Baroque", "Color Field Painting",
    "Contemporary Realism", "Cubism", "Early Renaissance", "Expressionism", "Fauvism", "High Renaissance",
    "Impressionism", "Mannerism Late Renaissance", "Minimalism", "Naive Art Primitivism", "New Realism",
    "Northern Renaissance", "Pointillism", "Pop Art", "Post Impressionism", "Realism", "Rococo", "Romanticism",
    "Symbolism", "Synthetic Cubism", "Ukiyo e"
]
dataset = TensorDataset(tensor_folder, class_names)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)

class LogisticRegression(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(LogisticRegression, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

logistic_regression = LogisticRegression(512, len(class_names)).to(device).half()

for param in model.parameters():
    param.requires_grad = False

optimizer = optim.SGD(logistic_regression.parameters(), lr=1e-2, momentum=0.9)
criterion = nn.CrossEntropyLoss()

epochs = 20
best_accuracy = 0

for epoch in range(epochs):
    logistic_regression.train()
    train_loss = 0
    correct = 0
    total = 0

    for images, labels in tqdm(dataloader):
        images, labels = images.to(device), labels.to(device)

        with torch.no_grad():
            features = model.encode_image(images)

        outputs = logistic_regression(features)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, preds = outputs.max(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_accuracy = correct / total * 100
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")

  0%|          | 0/30 [00:00<?, ?it/s]<ipython-input-73-bfbd862109f2>:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensor = torch.load(tensor_path)  # Load preprocessed

Epoch 1/20, Loss: 91.5566, Train Accuracy: 16.44%


100%|██████████| 30/30 [00:06<00:00,  4.84it/s]


Epoch 2/20, Loss: 73.1992, Train Accuracy: 40.39%


100%|██████████| 30/30 [00:06<00:00,  4.84it/s]


...

Epoch 18/20, Loss: 36.8867, Train Accuracy: 62.62%


100%|██████████| 30/30 [00:06<00:00,  4.78it/s]


Epoch 19/20, Loss: 36.2803, Train Accuracy: 62.59%


100%|██████████| 30/30 [00:06<00:00,  4.83it/s]

Epoch 20/20, Loss: 36.0703, Train Accuracy: 63.18%


In [77]:
#로지스틱 회귀 test데이터셋 평가 코드
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm

def evaluate_model(model, dataloader, class_names):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)

            features = model.encode_image(images)

            outputs = logistic_regression(features)
            preds = outputs.argmax(dim=1).cpu().numpy()

            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    print(f"Collected {len(all_preds)} predictions and {len(all_labels)} labels.")

    if not all_preds or not all_labels:
        raise ValueError("No predictions or labels collected. Check the dataset and dataloader!")

    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))

test_tensor_folder = "/content/dataset/processed_dataset"
test_dataset = TensorDataset(test_tensor_folder, class_names)
test_dataloader = DataLoader(test_dataset, batch_size=256, shuffle=False)

evaluate_model(model, test_dataloader, class_names)

Evaluating:   0%|          | 0/62 [00:00<?, ?it/s]<ipython-input-73-bfbd862109f2>:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensor = torch.load(tensor_path)  # Load 

Collected 15860 predictions and 15860 labels.
Accuracy: 52.07%

Classification Report:
                            precision    recall  f1-score   support

    Abstract Expressionism       0.63      0.56      0.59       512
           Action painting       0.67      0.27      0.39        22
         Analytical Cubism       0.79      0.52      0.63        21
        Art Nouveau Modern       0.58      0.48      0.52       841
                   Baroque       0.64      0.55      0.59       835
      Color Field Painting       0.65      0.77      0.70       312
      Contemporary Realism       0.15      0.67      0.25        96
                    Cubism       0.46      0.61      0.53       423
         Early Renaissance       0.49      0.74      0.59       280
             Expressionism       0.54      0.28      0.37      1286
                   Fauvism       0.19      0.69      0.30       182
          High Renaissance       0.42      0.43      0.42       271
             Impressionism  

###wikiart prompting